# Plotly 視覺化管理與 Notebook 容器處理

這個 Notebook 演示如何在 Jupyter 環境中正確管理 Plotly 視覺化，包括處理 DOM 變化、清理資源和確保視覺化在 Notebook 容器變化時的穩定性。

## 功能特色
- 使用 MutationObserver 監控 DOM 變化
- 自動清理已移除的 Plotly 視覺化
- 處理 Notebook 容器和輸出單元格的變化
- 確保視覺化資源的正確管理

In [1]:
# 導入必要的庫
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
from IPython.display import HTML, display
import json

## 1. 設置 Plotly 觀察器

建立 MutationObserver 來監控與 Plotly 視覺化相關的 DOM 變化。這確保當視覺化元素被移除時，我們可以適當地清理資源。

In [2]:
def create_plotly_observer_script():
    """
    建立 JavaScript 程式碼來設置 MutationObserver，
    用於監控 Plotly 視覺化的 DOM 變化
    """
    script = """
    <script>
    function setupPlotlyObserver(gd) {
        // 建立 MutationObserver 來監控顯示狀態變化
        var observer = new MutationObserver(function(mutations) {
            mutations.forEach(function(mutation) {
                var target = mutation.target;
                var display = window.getComputedStyle(target).display;
                
                // 檢查元素是否被隱藏或移除
                if (!display || display === 'none') {
                    console.log('Plotly 元素已被移除，開始清理:', gd);
                    
                    // 清理 Plotly 實例
                    if (typeof Plotly !== 'undefined' && Plotly.purge) {
                        Plotly.purge(gd);
                    }
                    
                    // 停止觀察
                    observer.disconnect();
                }
            });
        });
        
        // 觀察元素及其父元素的屬性變化
        observer.observe(gd, {
            attributes: true,
            attributeFilter: ['style', 'class'],
            subtree: true
        });
        
        // 觀察父容器的子元素變化
        if (gd.parentNode) {
            observer.observe(gd.parentNode, {
                childList: true,
                subtree: true
            });
        }
        
        return observer;
    }
    
    // 全域函數，用於設置新的 Plotly 圖表
    window.setupPlotlyChart = function(gd) {
        return setupPlotlyObserver(gd);
    };
    </script>
    """
    return script

# 執行基礎設置
display(HTML(create_plotly_observer_script()))
print("✅ Plotly 觀察器設置完成")

✅ Plotly 觀察器設置完成


## 2. 處理 Notebook 容器變化

使用 MutationObserver 檢測 Notebook 容器的變化，並處理 Notebook 單元格的移除事件。這確保當整個 Notebook 結構發生變化時，Plotly 視覺化能正確響應。

In [3]:
def create_notebook_container_observer():
    """
    建立監控 Notebook 容器變化的觀察器
    """
    script = """
    <script>
    function setupNotebookObserver() {
        // 尋找 Notebook 容器
        var notebookContainer = document.querySelector('#notebook-container') || 
                               document.querySelector('.notebook-container') ||
                               document.querySelector('[data-jp-notebook]');
        
        if (notebookContainer) {
            console.log('找到 Notebook 容器，設置觀察器');
            
            var notebookObserver = new MutationObserver(function(mutations) {
                mutations.forEach(function(mutation) {
                    // 檢查移除的節點
                    if (mutation.removedNodes.length > 0) {
                        mutation.removedNodes.forEach(function(node) {
                            if (node.nodeType === Node.ELEMENT_NODE) {
                                // 尋找被移除節點中的 Plotly 圖表
                                var plotlyElements = node.querySelectorAll ? 
                                    node.querySelectorAll('.plotly-graph-div') : [];
                                
                                // 清理找到的 Plotly 元素
                                for (var i = 0; i < plotlyElements.length; i++) {
                                    console.log('清理被移除的 Plotly 圖表:', plotlyElements[i]);
                                    if (typeof Plotly !== 'undefined' && Plotly.purge) {
                                        Plotly.purge(plotlyElements[i]);
                                    }
                                }
                            }
                        });
                    }
                });
            });
            
            // 觀察 Notebook 容器的子元素變化
            notebookObserver.observe(notebookContainer, {
                childList: true,
                subtree: true
            });
            
            console.log('Notebook 容器觀察器設置完成');
            return notebookObserver;
        } else {
            console.warn('未找到 Notebook 容器');
            return null;
        }
    }
    
    // 設置 Notebook 觀察器
    window.notebookObserver = setupNotebookObserver();
    </script>
    """
    return script

# 執行 Notebook 容器觀察器設置
display(HTML(create_notebook_container_observer()))
print("✅ Notebook 容器觀察器設置完成")

✅ Notebook 容器觀察器設置完成


## 3. 處理輸出單元格清理

使用 MutationObserver 檢測當前輸出單元格的清理操作，並相應地管理 Plotly 視覺化。這確保當單元格輸出被清理時，相關的視覺化資源也得到正確處理。

In [4]:
def create_output_cell_observer():
    """
    建立監控輸出單元格清理的觀察器
    """
    script = """
    <script>
    function setupOutputCellObserver() {
        // 尋找當前輸出區域
        var outputElements = document.querySelectorAll('.output, .jp-OutputArea, [data-mime-type]');
        var observers = [];
        
        outputElements.forEach(function(outputEl) {
            var outputObserver = new MutationObserver(function(mutations) {
                mutations.forEach(function(mutation) {
                    // 檢查是否有子節點被移除
                    if (mutation.removedNodes.length > 0) {
                        mutation.removedNodes.forEach(function(node) {
                            if (node.nodeType === Node.ELEMENT_NODE) {
                                // 檢查移除的節點是否包含 Plotly 圖表
                                var plotlyDivs = [];
                                
                                if (node.classList && node.classList.contains('plotly-graph-div')) {
                                    plotlyDivs.push(node);
                                } else if (node.querySelectorAll) {
                                    var foundDivs = node.querySelectorAll('.plotly-graph-div');
                                    plotlyDivs = plotlyDivs.concat(Array.from(foundDivs));
                                }
                                
                                // 清理找到的 Plotly 圖表
                                plotlyDivs.forEach(function(div) {
                                    console.log('輸出清理：清理 Plotly 圖表', div);
                                    if (typeof Plotly !== 'undefined' && Plotly.purge) {
                                        try {
                                            Plotly.purge(div);
                                        } catch (e) {
                                            console.warn('清理 Plotly 圖表時出錯:', e);
                                        }
                                    }
                                });
                            }
                        });
                    }
                    
                    // 檢查整個輸出區域是否被清空
                    if (mutation.type === 'childList' && 
                        outputEl.children.length === 0) {
                        console.log('輸出區域已清空');
                    }
                });
            });
            
            // 觀察輸出元素的子節點變化
            outputObserver.observe(outputEl, {
                childList: true,
                subtree: true
            });
            
            observers.push(outputObserver);
        });
        
        console.log('設置了', observers.length, '個輸出單元格觀察器');
        return observers;
    }
    
    // 設置輸出單元格觀察器
    window.outputObservers = setupOutputCellObserver();
    </script>
    """
    return script

# 執行輸出單元格觀察器設置
display(HTML(create_output_cell_observer()))
print("✅ 輸出單元格觀察器設置完成")

✅ 輸出單元格觀察器設置完成


## 4. 生成視覺化

建立和顯示 Plotly 圖表，確保在視覺化被移除時進行適當的清理。這個部分演示如何創建具有自動清理功能的 Plotly 視覺化。

In [5]:
def create_enhanced_plotly_chart(title="示例圖表", chart_type="scatter"):
    """
    建立具有增強清理功能的 Plotly 圖表
    """
    # 生成示例數據
    np.random.seed(42)
    x = np.linspace(0, 10, 100)
    y1 = np.sin(x) + np.random.normal(0, 0.1, 100)
    y2 = np.cos(x) + np.random.normal(0, 0.1, 100)
    
    if chart_type == "scatter":
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y1, mode='lines+markers', name='Sin(x) + noise'))
        fig.add_trace(go.Scatter(x=x, y=y2, mode='lines+markers', name='Cos(x) + noise'))
        
    elif chart_type == "subplots":
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('散點圖', '長條圖', '熱力圖', '3D 曲面'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"type": "surface"}]]
        )
        
        # 散點圖
        fig.add_trace(go.Scatter(x=x, y=y1, name='數據1'), row=1, col=1)
        
        # 長條圖
        categories = ['A', 'B', 'C', 'D', 'E']
        values = np.random.rand(5) * 100
        fig.add_trace(go.Bar(x=categories, y=values, name='分類數據'), row=1, col=2)
        
        # 熱力圖
        z = np.random.rand(10, 10)
        fig.add_trace(go.Heatmap(z=z, name='熱力圖'), row=2, col=1)
        
        # 3D 曲面
        x_3d = np.linspace(-5, 5, 20)
        y_3d = np.linspace(-5, 5, 20)
        X, Y = np.meshgrid(x_3d, y_3d)
        Z = np.sin(np.sqrt(X**2 + Y**2))
        fig.add_trace(go.Surface(x=X, y=Y, z=Z, name='3D 曲面'), row=2, col=2)
        
    fig.update_layout(
        title=title,
        showlegend=True,
        height=600 if chart_type == "subplots" else 400
    )
    
    # 顯示圖表並設置觀察器
    config = {
        'displayModeBar': True,
        'displaylogo': False,
        'modeBarButtonsToRemove': ['pan2d', 'lasso2d']
    }
    
    # 顯示圖表
    fig.show(config=config)
    
    # 添加自定義清理腳本
    cleanup_script = """
    <script>
    // 等待 Plotly 圖表渲染完成
    setTimeout(function() {
        var plotlyDivs = document.querySelectorAll('.plotly-graph-div');
        var latestDiv = plotlyDivs[plotlyDivs.length - 1];
        
        if (latestDiv && window.setupPlotlyChart) {
            console.log('為最新的 Plotly 圖表設置觀察器');
            window.setupPlotlyChart(latestDiv);
        }
    }, 500);
    </script>
    """
    
    display(HTML(cleanup_script))
    
    return fig

# 示例 1：基本散點圖
print("🎨 建立基本散點圖")
fig1 = create_enhanced_plotly_chart("Josephson 結響應示例", "scatter")

🎨 建立基本散點圖


In [6]:
# 示例 2：複雜的子圖佈局
print("\n🎨 建立複雜的子圖佈局")
fig2 = create_enhanced_plotly_chart("多重視覺化面板", "subplots")


🎨 建立複雜的子圖佈局


In [7]:
# 示例 3：Josephson 結特定的視覺化
def create_josephson_visualization():
    """
    建立 Josephson 結的專門視覺化
    """
    # 模擬 Josephson 結數據
    phi_ext = np.linspace(-2e-4, 0, 500)
    
    # 完整模型參數
    Ic = 1.0e-6
    phi_0 = np.pi / 4
    f = 5e4
    T = 0.8
    k = -0.01
    r = 5e-3
    C = 10.0e-6
    d = -10.0e-3
    
    # 計算電流響應
    phase_term = 2 * np.pi * f * (phi_ext - d) - phi_0
    sin_half = np.sin(phase_term / 2)
    denominator_arg = np.maximum(1 - T * sin_half**2, 1e-12)
    
    I_full = (Ic * np.sin(phase_term) / np.sqrt(denominator_arg) + 
              k * (phi_ext - d)**2 + r * (phi_ext - d) + C)
    
    I_simple = (Ic * np.sin(phase_term) + 
                k * (phi_ext - d)**2 + r * (phi_ext - d) + C)
    
    # 添加雜訊
    noise_level = 2e-7
    I_full_noisy = I_full + noise_level * np.random.normal(size=len(phi_ext))
    I_simple_noisy = I_simple + noise_level * np.random.normal(size=len(phi_ext))
    
    # 建立比較圖
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            '完整非線性模型 vs 簡化模型',
            '電流差異分析', 
            '相位響應',
            '頻譜分析'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # 子圖 1：模型比較
    fig.add_trace(
        go.Scatter(x=phi_ext*1e6, y=I_full_noisy*1e6, 
                  name='完整模型', line=dict(color='blue')), 
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=phi_ext*1e6, y=I_simple_noisy*1e6, 
                  name='簡化模型', line=dict(color='red', dash='dash')), 
        row=1, col=1
    )
    
    # 子圖 2：差異分析
    difference = (I_full - I_simple) * 1e6
    fig.add_trace(
        go.Scatter(x=phi_ext*1e6, y=difference, 
                  name='模型差異', line=dict(color='green')), 
        row=1, col=2
    )
    
    # 子圖 3：相位響應
    fig.add_trace(
        go.Scatter(x=phi_ext*1e6, y=phase_term, 
                  name='相位項', line=dict(color='purple')), 
        row=2, col=1
    )
    
    # 子圖 4：FFT 頻譜
    from scipy.fft import fft, fftfreq
    
    # 計算 FFT
    fft_values = np.abs(fft(I_full_noisy))
    freqs = fftfreq(len(I_full_noisy), d=(phi_ext[1] - phi_ext[0]))
    
    # 只顯示正頻率部分
    positive_freqs = freqs[:len(freqs)//2]
    positive_fft = fft_values[:len(fft_values)//2]
    
    fig.add_trace(
        go.Scatter(x=positive_freqs, y=positive_fft, 
                  name='FFT 功率譜', line=dict(color='orange')), 
        row=2, col=2
    )
    
    # 更新佈局
    fig.update_xaxes(title_text="外部磁通 (μWb)", row=1, col=1)
    fig.update_xaxes(title_text="外部磁通 (μWb)", row=1, col=2)
    fig.update_xaxes(title_text="外部磁通 (μWb)", row=2, col=1)
    fig.update_xaxes(title_text="頻率", row=2, col=2)
    
    fig.update_yaxes(title_text="電流 (μA)", row=1, col=1)
    fig.update_yaxes(title_text="電流差異 (μA)", row=1, col=2)
    fig.update_yaxes(title_text="相位 (rad)", row=2, col=1)
    fig.update_yaxes(title_text="FFT 幅度", row=2, col=2)
    
    fig.update_layout(
        title="Josephson 結全面分析",
        height=800,
        showlegend=True
    )
    
    return fig

print("\n🔬 建立 Josephson 結專門視覺化")
fig3 = create_josephson_visualization()
fig3.show()

# 為這個圖表也設置觀察器
cleanup_script = """
<script>
setTimeout(function() {
    var plotlyDivs = document.querySelectorAll('.plotly-graph-div');
    var latestDiv = plotlyDivs[plotlyDivs.length - 1];
    
    if (latestDiv && window.setupPlotlyChart) {
        console.log('為 Josephson 視覺化設置觀察器');
        window.setupPlotlyChart(latestDiv);
    }
}, 500);
</script>
"""

display(HTML(cleanup_script))
print("✅ Josephson 結視覺化完成")


🔬 建立 Josephson 結專門視覺化


✅ Josephson 結視覺化完成


## 5. 測試清理功能

這個部分提供了一些工具來測試 Plotly 視覺化的清理功能是否正常工作。

In [8]:
def test_cleanup_functionality():
    """
    測試 Plotly 清理功能
    """
    test_script = """
    <script>
    function testPlotlyCleanup() {
        console.log('開始測試 Plotly 清理功能...');
        
        // 檢查當前頁面的 Plotly 圖表數量
        var plotlyDivs = document.querySelectorAll('.plotly-graph-div');
        console.log('找到', plotlyDivs.length, '個 Plotly 圖表');
        
        // 檢查觀察器是否設置
        console.log('Notebook 觀察器:', window.notebookObserver ? '已設置' : '未設置');
        console.log('輸出觀察器:', window.outputObservers ? 
                   ('已設置 ' + window.outputObservers.length + ' 個') : '未設置');
        
        // 模擬清理第一個圖表（如果存在）
        if (plotlyDivs.length > 0) {
            var firstDiv = plotlyDivs[0];
            console.log('模擬清理第一個圖表...');
            
            if (typeof Plotly !== 'undefined' && Plotly.purge) {
                try {
                    Plotly.purge(firstDiv);
                    console.log('✅ 第一個圖表清理成功');
                } catch (e) {
                    console.error('❌ 清理失敗:', e);
                }
            } else {
                console.warn('⚠️ Plotly.purge 不可用');
            }
        }
        
        return {
            totalCharts: plotlyDivs.length,
            notebookObserver: !!window.notebookObserver,
            outputObservers: window.outputObservers ? window.outputObservers.length : 0
        };
    }
    
    // 執行測試
    window.testResults = testPlotlyCleanup();
    console.log('測試結果:', window.testResults);
    </script>
    """
    
    display(HTML(test_script))
    print("🧪 Plotly 清理功能測試完成，請查看瀏覽器控制台獲取詳細結果")

# 執行測試
test_cleanup_functionality()

🧪 Plotly 清理功能測試完成，請查看瀏覽器控制台獲取詳細結果


## 總結與最佳實踐

這個 Notebook 展示了在 Jupyter 環境中管理 Plotly 視覺化的完整解決方案：

### 🎯 主要功能
1. **自動資源清理**：使用 MutationObserver 自動檢測和清理被移除的 Plotly 圖表
2. **容器變化處理**：監控 Notebook 容器變化，確保視覺化穩定性
3. **輸出清理管理**：當輸出單元格被清理時自動處理相關資源
4. **增強視覺化**：提供具有自動清理功能的 Plotly 圖表建立工具

### 💡 最佳實踐
1. **總是設置觀察器**：在建立 Plotly 圖表前先設置相應的觀察器
2. **及時清理資源**：確保在不需要時清理 Plotly 實例以避免記憶體洩漏
3. **錯誤處理**：在清理操作中加入適當的錯誤處理機制
4. **測試功能**：定期測試清理功能確保其正常工作

### 🔧 使用建議
- 在每個新的 Notebook 中都執行觀察器設置代碼
- 對於複雜的視覺化，考慮使用自定義清理函數
- 在開發環境中啟用控制台日誌以監控清理過程
- 定期檢查瀏覽器記憶體使用情況